# Exercise 03.b: Stability of the oscillation equation

**Due:** _15 December 2020_

**Names:** _Enter your names here._

Considering the following oscillation equation
$$\frac{{\rm d}q}{{\rm d}t}+i f q = 0$$
where $f$ is the Coriolis parameter. Its analytic solution is $q=q_{0} {\rm e}^{-ift}$.

1. Discretize the oscillation equation using forward Euler, backward Euler and leap-frog schemes and analyse their stability, respectively.

1. With initial value $q_0 = {\rm e}^{i{\pi}/4}$, and $f=10^{-4}s^{-1}$ and $\Delta t=100s$, attempt to solve the oscillation equation by forward Euler, backward Euler and leap-frog schemes, respectively. Integrate the numerical equations for about ten periods of oscillation and compare the numerical solutions with analytic solution.

1. Take $\Delta t=200, 500 \text{ and } 1000s$, repeat the integration, and compare them with analytic solution.
